In [7]:
import requests
from bs4 import BeautifulSoup
import time
import random
import numpy as np
import logging
import csv  # Mengimpor csv untuk menyimpan data ke file CSV.

# Konfigurasi logging
logging.basicConfig(filename='scraping.log', level=logging.INFO, format='%(asctime)s - %(levelname)s - %(message)s')

# Kode warna
GREEN = "\033[92m"  # Warna hijau
RED = "\033[91m"    # Warna merah
RESET = "\033[0m"   # Reset ke warna default

def scrape_kompas(pages: int, csv_filename: str) -> None:
    """
    Melakukan scraping berita dari situs Kompas dan menyimpan hasilnya ke file CSV.

    Args:
        pages (int): Jumlah halaman yang akan di-scrape.
        csv_filename (str): Nama file CSV untuk menyimpan hasil.

    Returns:
        None
    """
    # Header CSV
    fieldnames = ['judul', 'topik', 'sub_topik', 'topik_pilihan', 'tanggal_waktu_publish', 'redaksi', 'advetorial', 'isi_berita', 'link', 'topik_pilihan_link']
    
    # Membuka file CSV dalam mode append ('a') dan menulis header jika file baru
    with open(csv_filename, mode='a', newline='', encoding='utf-8') as file:
        writer = csv.DictWriter(file, fieldnames=fieldnames)
        
        # Menulis header hanya jika file baru (dengan mengecek apakah filenya kosong)
        if file.tell() == 0:
            writer.writeheader()
        
        # Loop untuk setiap halaman dari 1 hingga pages
        for i in range(1, pages + 1):
            try:
                url = f"https://indeks.kompas.com/?site=all&page={i}"
                response = requests.get(url)
                
                soup = BeautifulSoup(response.text, 'html.parser')
                links = soup.find_all('a', class_='article-link')
                links = [link.get('href') for link in links]
                
                # Loop untuk setiap link artikel yang ditemukan
                for j, link in enumerate(links):
                    try:
                        response_news = BeautifulSoup(requests.get(link).text, 'html.parser')

                        # Cek apakah artikel adalah advertorial
                        try:
                            advetorial = response_news.find('div', class_='kcm__header__advertorial').get_text()
                        except:
                            advetorial = ''

                        # Mengambil topik dari breadcrumb
                        try:
                            topic_tags = response_news.find_all('li', class_='breadcrumb__item')
                            topics = [tag.find('span').get_text() for tag in topic_tags]
                            topik = topics[1]
                        except:
                            topik = ''

                        # Mengambil sub-topik dari breadcrumb
                        try:
                            sub_topik = topics[2]
                        except:
                            sub_topik = ''

                        # Mengambil topik pilihan jika ada
                        try:
                            topik_pilihan = response_news.find('div', class_='topicSubtitle').find('a').get_text()
                            topik_pilihan_link = response_news.find('div', class_='topicSubtitle').find('a').get('href')
                        except:
                            topik_pilihan = ''
                            topik_pilihan_link = ''

                        # Mengambil judul artikel
                        try:
                            judul = response_news.find('h1', class_='read__title').get_text()
                        except:
                            judul = ''

                        # Mengambil tanggal dan waktu publikasi
                        try:
                            tanggal_waktu_publish = response_news.find('div', class_='read__time').get_text().split(' - ')[1]
                        except:
                            tanggal_waktu_publish = ''

                        # Mengambil nama redaksi yang menulis artikel
                        try:
                            redaksi_tag = response_news.find('div', class_='credit-title-name').find_all('h6')
                            redaksi = [penulis.get_text() for penulis in redaksi_tag]
                            redaksi = ' '.join(redaksi)
                        except:
                            redaksi = ''

                        # Mengambil isi berita
                        try:
                            konteks_tag = response_news.find('div', class_='read__content').find_all('p')
                            isi_berita = ' '.join([konteks.get_text() for konteks in konteks_tag])
                        except:
                            isi_berita = ''

                        # Simpan hasil scraping ke dalam dictionary
                        result = {
                            'judul': judul,
                            'topik': topik,
                            'sub_topik': sub_topik,
                            'topik_pilihan': topik_pilihan,
                            'tanggal_waktu_publish': tanggal_waktu_publish,
                            'redaksi': redaksi,
                            'advetorial': advetorial,
                            'isi_berita': isi_berita,
                            'link': link,
                            'topik_pilihan_link': topik_pilihan_link
                        }

                        # Menulis hasil ke file CSV
                        writer.writerow(result)

                        random_delay = random.uniform(0.1, 1)
                        time.sleep(random_delay)
                    except:
                        print(f'{RED}ERROR - page {i} link {j + 1}  = {link}')
                        logging.info(f'ERROR - page {i} link {j + 1}  = {link}')
                        random_delay = random.uniform(0.1, 1)
                        time.sleep(random_delay)
                print()

            except Exception as e:
                print(f'{RED}ERROR - page {i}\n')
                logging.info(f'ERROR - page {i}\n')
                continue

# Contoh pemanggilan fungsi
scrape_kompas(9, 'scraping_result.csv')
